In [1]:
import scanpy as sc
import stlearn as st
import numpy as np
import csv
import pickle
import matplotlib
import math
import json
from itertools import combinations
import pandas as pd
import matplotlib
# from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite

spot_diameter = 89.43 #pixels

In [2]:
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    adata_h5 = st.Read10X(path=args.data_path, count_file='filtered_feature_bc_matrix.h5') #count_file=args.data_name+'_filtered_feature_bc_matrix.h5' )
    print(adata_h5)
    sc.pp.filter_genes(adata_h5, min_cells=1)
    print(adata_h5)
    gene_ids = list(adata_h5.var_names)
    coordinates = adata_h5.obsm['spatial']
    cell_barcode = np.array(adata_h5.obs.index)
    temp = adata_h5.X

NameError: name 'args' is not defined

In [3]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)


# # Load pathologist's label
# data_name = 'PDAC_64630'
# if data_name == 'PDAC_64630':
#     pathologist_label_file='/Users/victoriagao/local_docs/NEST/input/pathologist_annot/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
#     pathologist_label=[]
#     with open(pathologist_label_file) as file:
#         csv_file = csv.reader(file, delimiter=",")
#         for line in csv_file:
#             pathologist_label.append(line)	
    	
#     barcode_type=dict() # record the type (annotation) of each spot (barcode)
#     for i in range (1, len(pathologist_label)):
#         barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]



In [4]:
# Load subtype label
subtype_label_file='/Users/victoriagao/local_docs/schwartz_data/PDAC_64630_subtype.csv'
subtype_label=[]
with open(subtype_label_file) as file:
    csv_file = csv.reader(file, delimiter=",")
    for line in csv_file:
        subtype_label.append(line)

barcode_subtype=dict()
for i in range(1,len(subtype_label)):
    barcode_subtype[subtype_label[i][0]]= subtype_label[i][1]

In [5]:
barcode_subtype

{'AAACCGGGTAGGTACC-1': 'BasalB',
 'AAACCGTTCGTCCAGG-1': 'Mixed',
 'AAACGAGACGGTTGAT-1': 'Mixed',
 'AAACTGCTGGCTCCAA-1': 'Mixed',
 'AAAGACCCAAGTCGCG-1': 'BasalB',
 'AAAGGCTCTCGCGCCG-1': 'ClassicB',
 'AAAGGGATGTAGCAAG-1': 'Mixed',
 'AAAGGGCAGCTTGAAT-1': 'ClassicB',
 'AAAGTAGCATTGCTCA-1': 'Mixed',
 'AAAGTGTGATTTATCT-1': 'BasalB',
 'AAATACCTATAAGCAT-1': 'Mixed',
 'AAATAGGGTGCTATTG-1': 'Mixed',
 'AAATCGTGTACCACAA-1': 'BasalB',
 'AAATGGTCAATGTGCC-1': 'BasalB',
 'AAATTAACGGGTAGCT-1': 'Mixed',
 'AAATTGATAGTCCTTT-1': 'Mixed',
 'AAATTTGCGGGTGTGG-1': 'Mixed',
 'AACAACTGGTAGTTGC-1': 'BasalB',
 'AACAATTACTCTACGC-1': 'BasalB',
 'AACACACGCTCGCCGC-1': 'Mixed',
 'AACAGGATGGGCCGCG-1': 'BasalB',
 'AACCAAGACTTCTCTG-1': 'BasalB',
 'AACCCAGAGACGGAGA-1': 'BasalB',
 'AACCGCTAAGGGATGC-1': 'Mixed',
 'AACCTTTAAATACGGT-1': 'ClassicA',
 'AACGATAATGCCGTAG-1': 'Mixed',
 'AACGCGGTCTCCAGCC-1': 'Mixed',
 'AACGCTGTTGCTGAAA-1': 'ClassicA',
 'AACGGACGTACGTATA-1': 'Mixed',
 'AACGGCCATCTCCGGT-1': 'Mixed',
 'AACGTACTGTGGGTAC

In [6]:
len(barcode_subtype)

1406

In [7]:
# Make barcode_info
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
barcode_info.append(["cell_code", "x-coordinate", "y-coordinate", "component", "color", "barcode_number"]) # add title
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0, 0, 0, 0]) # last entries will hold the component number later
    i=i+1


barcode_coordinate_dict = {item[0]: [item[1], item[2]] for item in barcode_info}

combined_barcode = {}
for key in barcode_subtype:
    if key in barcode_coordinate_dict:
        combined_barcode[key] = [barcode_subtype[key]] + barcode_coordinate_dict[key]


combined_barcode_list = []

for key, values in combined_barcode.items():
    temp_dict = {'barcode': key, 'type': values[0], 'x': values[1], 'y': values[2]}
    combined_barcode_list.append(temp_dict)

In [8]:
combined_barcode_list

[{'barcode': 'AAACCGGGTAGGTACC-1', 'type': 'BasalB', 'x': 9283, 'y': 12018},
 {'barcode': 'AAACCGTTCGTCCAGG-1', 'type': 'Mixed', 'x': 11004, 'y': 10622},
 {'barcode': 'AAACGAGACGGTTGAT-1', 'type': 'Mixed', 'x': 8053, 'y': 6963},
 {'barcode': 'AAACTGCTGGCTCCAA-1', 'type': 'Mixed', 'x': 9785, 'y': 8146},
 {'barcode': 'AAAGACCCAAGTCGCG-1', 'type': 'BasalB', 'x': 3750, 'y': 10057},
 {'barcode': 'AAAGGCTCTCGCGCCG-1', 'type': 'ClassicB', 'x': 11516, 'y': 9329},
 {'barcode': 'AAAGGGATGTAGCAAG-1', 'type': 'Mixed', 'x': 6161, 'y': 8658},
 {'barcode': 'AAAGGGCAGCTTGAAT-1', 'type': 'ClassicB', 'x': 6176, 'y': 12230},
 {'barcode': 'AAAGTAGCATTGCTCA-1', 'type': 'Mixed', 'x': 10837, 'y': 12111},
 {'barcode': 'AAAGTGTGATTTATCT-1', 'type': 'BasalB', 'x': 9600, 'y': 5468},
 {'barcode': 'AAATACCTATAAGCAT-1', 'type': 'Mixed', 'x': 10129, 'y': 7946},
 {'barcode': 'AAATAGGGTGCTATTG-1', 'type': 'Mixed', 'x': 11161, 'y': 6949},
 {'barcode': 'AAATCGTGTACCACAA-1', 'type': 'BasalB', 'x': 9617, 'y': 9238},
 {'ba

In [9]:
# calculate euclidean distance between two points
def euclidean_distance(p1, p2):
    return math.sqrt((p1['x'] - p2['x'])**2 + (p1['y'] - p2['y'])**2)

max_distance = 0
max_distance_barcodes = ()

# Iterate through all pairs of dictionaries and calculate distances - Max
for i, dict1 in enumerate(combined_barcode_list):
    for j, dict2 in enumerate(combined_barcode_list[i + 1:], start=i + 1):
        distance = euclidean_distance(dict1, dict2)
        if distance > max_distance:
            max_distance = distance
            max_distance_barcodes = (dict1['barcode'], dict2['barcode'])


# Initialize the minimum distance with a large value
min_distance = float('inf')

# Iterate through all pairs of dictionaries and calculate distances
for i, dict1 in enumerate(combined_barcode_list):
    for j, dict2 in enumerate(combined_barcode_list[i + 1:], start=i + 1):
        distance = euclidean_distance(dict1, dict2)
        if distance < min_distance:
            min_distance = distance
            closest_barcodes = (dict1['barcode'], dict2['barcode'])

In [10]:
print(min_distance)
############# Juse to be safe, use min_distance + 1 or 2 to find all neighbors

198.0


In [11]:
# Find connected components
G = nx.Graph()

# Add nodes to the graph
for component in combined_barcode_list:
    G.add_node(component['barcode'], x=component['x'], y=component['y'], type=component['type'])

# Define proximity threshold and type matching condition
proximity_threshold = 200

# Iterate through components and add edges based on proximity and type
for i, component1 in enumerate(combined_barcode_list):
    for j, component2 in enumerate(combined_barcode_list[i + 1:], start=i + 1):
        distance = ((component1['x'] - component2['x'])**2 + (component1['y'] - component2['y'])**2)**0.5
        if distance <= proximity_threshold and component1['type'] == component2['type']:
            G.add_edge(component1['barcode'], component2['barcode'])
            # print("this"+component1['id']+component2['id'])

# Print the edges representing the connections
# print("Edges:", list(G.edges()))

connected_components = list(nx.connected_components(G)) # gives a list of sets, each set is a connected clusterfiltered_sets = [s for s in list_of_sets if len(s) > 1]
filtered_connected_components = [s for s in connected_components if len(s) > 1] # has to be more than one spot


# Print the connected components
for idx, component_set in enumerate(filtered_connected_components, start=1):
    component_types = {component['type'] for barcode in component_set for component in combined_barcode_list if component['barcode'] == barcode}
    print(f"Connected Component {idx} (Type: {component_types}): {component_set}")

Connected Component 1 (Type: {'BasalB'}): {'CGCTTATTCCCGGTCG-1', 'CAAATCTCTCACAAGG-1', 'TTCGTACTCCAGAACG-1', 'TCCCAGGCTTAGCTAA-1', 'CGCAATTACTTTCGGT-1', 'TCCAGGCGAGTACGGT-1', 'TTACCATTGATTACCC-1', 'CGCAATTCTACAATAA-1', 'TCCCGGGTGTGCTGCT-1', 'GAGAGGGCGCGAGGTT-1', 'ATAAAGGCTCGGTCGT-1', 'CACAGCACCCACGGCA-1', 'TGATCAGGGAACTGCT-1', 'TGACGAATATTTCCCT-1', 'TCGAAATTTAGGACCA-1', 'CGGTTGACCTGGCATA-1', 'GATCTTTGCAGGGTAT-1', 'CAGTTCCGCGGGTCGA-1', 'CGCAATTAGGGTAATA-1', 'TCCGATTACATTGCCG-1', 'GGCGTCCTATCCGCTG-1', 'CTTCATTGTCAGTGGA-1', 'GTCACTCTCCAAATCT-1', 'TCACATCTTATCTGAT-1', 'CCATGGCCCTTGTACC-1', 'TGATCTCCGGCGCCAG-1', 'ATTGAAGATCTTAGTG-1', 'CCGGGTTCGAGGTTAC-1', 'AAGGAGCGGTTGGTGC-1', 'GCCGAAATTCCTACGT-1', 'CTTGCAACCGCCTCCT-1', 'TATGAAGACAGGTGCG-1', 'CCGGAATGGTTTCAGT-1', 'ATAAATATTAGCAGCT-1', 'GGCCCTCACCCACTTA-1', 'GAGGAATGGAGAGGTT-1', 'ATAAGTTACCGCGACG-1', 'CGGGCAGCTAAACCGC-1', 'CTACTCTAGGCCCGGC-1', 'TCAGGTTCTTTGAGAA-1', 'CGCCTGGCCTACGTAA-1', 'TCTGGGTAGCGCTCAT-1', 'ACTACCAGCTCTCTGG-1', 'TTCTTGTAAC

In [12]:
with open('/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_connected_subtypes/PDAC_64630_filtered_connected_subtype_components.pkl', 'wb') as file:
    pickle.dump(filtered_connected_components, file)

list